##### **This embedding gets the reply embeddings for english replies**

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config_hp
import helper.pandas_helper as pd_hp
import helper.embedding as emb_hp

2023-03-19 17:32:41.327210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 17:32:41.765203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel8/mvapich/gnu/2.3.5/lib:/N/soft/rhel8/gcc/9.3.0/lib64:/N/soft/rhel8/gcc/9.3.0/lib
2023-03-19 17:32:41.765243: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-19 17:32:45.010056: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic libr

##### **Get log files**

In [2]:
config = config_hp.config()
path = config['PATHS']

derived_path = path['derived_path']
all_replies = path['all_replies']

file_hp.create_folder(derived_path, 'reply_embedding')
reply_embedding_path = os.path.join(derived_path, 'reply_embedding')
reply_embedding_file = os.path.join(reply_embedding_path, 
                                    'all_reply_embedding.pkl.gz')
print(reply_embedding_file)

/N/slate/potem/data/derived/reply_embedding/all_reply_embedding.pkl.gz


##### **Get all replies in english**

In [3]:
df_replies = pd.read_pickle(all_replies)

In [4]:
df_replies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21427155 entries, 0 to 21427154
Data columns (total 9 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   replier_tweetid  object
 1   replier_userid   object
 2   poster_tweetid   object
 3   poster_userid    object
 4   tweet_language   object
 5   tweet_text       object
 6   tweet_time       object
 7   year             object
 8   campaign         object
dtypes: object(9)
memory usage: 1.4+ GB


In [5]:
df_replies.columns

Index(['replier_tweetid', 'replier_userid', 'poster_tweetid', 'poster_userid',
       'tweet_language', 'tweet_text', 'tweet_time', 'year', 'campaign'],
      dtype='object')

In [16]:
# df_replies['replier_tweetid'].unique()

In [6]:
df_eng = df_replies.loc[df_replies['tweet_language'] == 'en']
df_eng = df_eng.drop_duplicates()

In [7]:
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798186 entries, 0 to 21427154
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   replier_tweetid  798186 non-null  object
 1   replier_userid   798186 non-null  object
 2   poster_tweetid   798186 non-null  object
 3   poster_userid    798186 non-null  object
 4   tweet_language   798186 non-null  object
 5   tweet_text       798186 non-null  object
 6   tweet_time       798186 non-null  object
 7   year             798186 non-null  object
 8   campaign         798186 non-null  object
dtypes: object(9)
memory usage: 60.9+ MB


In [8]:
sentences = df_eng.head()['tweet_text'].tolist()

In [9]:
embeddings = emb_hp.get_sentence_embedding(sentences)

df_emb = pd.DataFrame(columns=['embeddings', 'replier_tweetid'])
df_emb['embeddings'] = embeddings
df_emb['replier_tweetid'] = df_eng['replier_tweetid']

df_emb.to_pickle(f'{reply_embedding_file}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##### **Get cosine of reply embeddings**

###### **Load config**

In [2]:
config = config_hp.config()
path = config['PATHS']
external_replies = path['external_reply']
emb_path = config['EMBEDDINGS_PATH']
reply_embedding = emb_path['reply_embedding']
english_reply_cosine = emb_path['reply_english_cosine']

###### **Read external replies and filter english replies**

In [3]:
df_replies = pd.read_pickle(external_replies)
df_eng = df_replies.loc[df_replies['tweet_language'] == 'en']

In [4]:
df_eng = df_eng.drop_duplicates()

df_eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798186 entries, 0 to 21427153
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   replier_tweetid  798186 non-null  int64  
 1   replier_userid   798186 non-null  object 
 2   poster_tweetid   798186 non-null  float64
 3   poster_userid    798186 non-null  float64
 4   tweet_language   798186 non-null  object 
 5   tweet_time       798186 non-null  object 
 6   year             798186 non-null  object 
 7   campaign         798186 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 54.8+ MB


In [5]:
df_eng = df_eng.drop_duplicates(subset=['replier_tweetid'], 
                                keep='last')

In [6]:
df_eng['replier_tweetid'].nunique()

795335

###### **Read embedding of all replies**

In [7]:
df_emb = pd.read_pickle(f'{reply_embedding}')

In [8]:
df_emb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 795833 entries, 0 to 795832
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   embeddings       795833 non-null  object
 1   replier_tweetid  795833 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 12.1+ MB


###### **Get only replies in english**

In [9]:
df_ex_emb = df_emb.loc[
    df_emb['replier_tweetid'].isin(
        df_eng['replier_tweetid'])]

In [10]:
df_ex_emb = df_ex_emb.merge(df_eng, on=['replier_tweetid'])

In [12]:
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine(vector_list):
    '''
    Get the cosine similarity of the vector list
    :param vector_list: list of embedding vectors
    
    :return list
    '''
    ref_comb = combinations(vector_list, 2)
    all_cosine = []
    
    for ref in ref_comb:
        cosine_ref_ref = round(cosine_similarity([ref[0]],
                                                 [ref[1]])[0][0],
                               2)
        all_cosine.append(cosine_ref_ref)
        
    return all_cosine

In [25]:
grps = df_ex_emb.groupby(['poster_tweetid'])

In [46]:
def get_reply_cosines(grps, save_path=None):
    '''
    Get the cosines of all reply combinations
    :param grps: the group of replies to poster tweet ids
    '''
    all_cosine = []
    for grp in grps:
        if len(grp[1]) > 1:
            cosine_value = get_cosine(grp[1]['embeddings'].tolist())
            all_cosine.append([grp[0], cosine_value])

    df = pd.DataFrame(data=all_reply_cosine, 
                  columns=['poster_tweetid', 'cosine']
                     )
    if save_path == None:
        return df
    
    df.to_pickle(save_path)

    
# get_reply_cosines(grps, english_reply_cosine)

###### **Read poster tweets with 2 or more replies in english and find number of tweets with less than 2**

In [47]:
df_cosine = pd.read_pickle(english_reply_cosine)

In [48]:
df_cosine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30109 entries, 0 to 30108
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   poster_tweetid  30109 non-null  float64
 1   cosine          0 non-null      object 
dtypes: float64(1), object(1)
memory usage: 470.6+ KB
